In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 52

In [ ]:
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from chromadb import PersistentClient
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import sqlite3

In [ ]:
conn = sqlite3.connect(r'C:\Users\HOME\Downloads\eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[]


In [ ]:
# Reading the columns of Table
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

In [ ]:
# Loading the Database Table inside a Pandas DataFrame
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
# Applying the above Function on the Entire Data
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
import random

random_df = df.sample(frac=0.3, random_state=42)

print(random_df.shape)

random_df.head()

(24749, 4)


,num,name,content,file_content
17262,9251120,maybe.this.time.(2014).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x89\x9a\x...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,b'PK\x03\x04\x14\x00\x00\x00\x08\x007\x8f\x99V...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x8f\x19\x...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\xaa\x99V...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,9408707,battlebots.(2015).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf4<\x9aV...,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."


In [ ]:
random_df.to_csv("randomsorted.csv",index=False,escapechar="\\")

In [ ]:
import os
os.getcwd()

'C:\\Users\\HOME\\Downloads'

In [ ]:
df1=pd.read_csv(r"C:\Users\HOME\Downloads\randomsorted.csv")

In [ ]:
print(df1.shape)
df1.head()

(24749, 4)


,num,name,content,file_content
0,9251120,maybe.this.time.(2014).eng.1cd,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x007\...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00[\...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
4,9408707,battlebots.(2015).eng.1cd,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."


In [ ]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def clean_subtitle(subtitle):
    # Remove timestamps
    clean_content = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n?', '', subtitle)

    # Remove other non-textual patterns
    clean_content = re.sub(r'<[^>]+>', '', clean_content)

    clean_content = re.sub(r"[^\w\s]", '', clean_content)

    clean_content = re.sub(r"[^\x00-\x7F]+", '', clean_content)

    clean_content = re.sub(r"\b\d+\s", '', clean_content)

    # Convert to lowercase
    clean_content = clean_content.lower()

    # Tokenize the subtitle content
    tokens = word_tokenize(clean_content)

    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words('english'))

    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words]

    # Join the filtered tokens back into a string
    clean_content = ' '.join(clean_tokens)

    return clean_content.strip()

In [ ]:
df1['cleaned_content'] = df1['file_content'].apply(clean_subtitle)

In [ ]:
df1.head()

,num,name,content,file_content,cleaned_content
0,9251120,maybe.this.time.(2014).eng.1cd,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch video online opensubtitles free browser ...
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x007\...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ...",oh know getting late dont wan na go home im hu...
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin...",timing subtitle uncontrollable lovebird team v...
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00[\...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea...",ethereal music apiopensubtitlesorg deprecated ...
4,9408707,battlebots.(2015).eng.1cd,b'PK\\x03\\x04\\x14\\x00\\x00\\x00\\x08\\x00\\...,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri...",chris oh minibots yelling oh leave little bot ...


### **4. (Must Implement) A very important step to improve the performance: Document Chunker.**

a. Consider the challenge of embedding large documents: Information Loss.

b. It is often not practical to embed an entire document as a single vector, particularly when dealing with long documents.

c. Solution: Divide a large document into smaller, more manageable chunks for embedding.

d. Another Problem: Let’s say we set the token window to be 500, then we’d expect each chunk to be just below 500 tokens. One common concern of this method is that we might accidentally cut off some important text between chunks, splitting up the context. To mitigate this, we can set overlapping windows with a specified amount of tokens to overlap so we have tokens shared between chunks.

In [ ]:
#df1.to_csv("cleaneddata.csv",index=False,escapechar="\\")

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv(r"C:\Users\HOME\Downloads\cleaneddata.csv")

In [ ]:
df.shape

(4034, 5)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Function to count tokens
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

# Iterate through data and count tokens for each file
for index, entry in df.iterrows():
    total_tokens = count_tokens(entry["cleaned_content"])
    print(f"File: {entry['name']}, Number of Tokens: {total_tokens}")

File: maybe.this.time.(2014).eng.1cd, Number of Tokens: 5335
File: down.the.shore.s01.e10.and.justice.for.all.(1992).eng.1cd, Number of Tokens: 1356
File: uncontrollably.fond.s01.e07.heartache.(2016).eng.1cd, Number of Tokens: 2425
File: screen.two.s13.e04.the.precious.blood.(1996).eng.1cd, Number of Tokens: 3588
File: battlebots.(2015).eng.1cd, Number of Tokens: 7743
File: csi.crime.scene.investigation.s08.e16.two.and.a.half.deaths.(2008).eng.1cd, Number of Tokens: 2998
File: royal.ashes.().eng.1cd, Number of Tokens: 5537
File: return.to.seoul.(2022).eng.1cd, Number of Tokens: 2503
File: idris.elba.king.of.speed.s01.e02.episode.1.2.(2013).eng.1cd, Number of Tokens: 3433
File: tooth.pari.when.love.bites.s01.e08.episode.1.8.(2023).eng.1cd, Number of Tokens: 1863
File: studio.one.s08.e30.the.arena.(1956).eng.1cd, Number of Tokens: 3623
File: love.life.s02.e10.epilogue.(2021).eng.1cd, Number of Tokens: 2011
File: scrubs.s03.e14.my.screwup.(2004).eng.1cd, Number of Tokens: 1416
File: devot

File: shameless.s09.e14.found.(2019).eng.1cd, Number of Tokens: 6258
File: bobs.burgers.s13.e13.stop.or.my.mom.will.sleuth.(2023).eng.1cd, Number of Tokens: 2284
File: its.beautiful.now.s01.e44.episode.1.44.(2022).eng.1cd, Number of Tokens: 3976
File: northern.exposure.s05.e20.a.wing.and.a.prayer.(1994).eng.1cd, Number of Tokens: 5384
File: narcos.s02.e02.cambalache.(2016).eng.1cd, Number of Tokens: 2325
File: 1899.s01.e05.the.calling.(2022).eng.1cd, Number of Tokens: 38
File: reign.s02.e10.mercy.(2014).eng.1cd, Number of Tokens: 2071
File: fruits.basket.s03.e05.i.mean.you.know.right.(2021).eng.1cd, Number of Tokens: 1299
File: iki.wo.hisomete.s01.e03.episode.1.3.(2021).eng.1cd, Number of Tokens: 1014
File: beverly.hills.90210.s06.e08.gypsies.cramps.and.fleas.(1995).eng.1cd, Number of Tokens: 2333
File: scrubs.s09.e05.our.mysteries.(2009).eng.1cd, Number of Tokens: 1566
File: sexify.s02.e03.episode.2.3.(2023).eng.1cd, Number of Tokens: 1712
File: murdoch.mysteries.s04.e01.tattered.and.

File: love.in.wolf.creek.(2022).eng.1cd, Number of Tokens: 4133
File: magnum.p.i.s05.e04.nsfw.(2023).eng.1cd, Number of Tokens: 3136
File: britains.most.evil.killers.s01.e08.dennis.nilsen.(2017).eng.1cd, Number of Tokens: 3231
File: classroom.of.the.elite.s02.e07.to.doubt.everything.or.to.believe.everything.are.two.equally.convenient.solutions.both.dispense.with.the.necessity.of.reflection.(2022).eng.1cd, Number of Tokens: 2559
File: futurama.s02.e07.a.head.in.the.polls.(1999).eng.1cd, Number of Tokens: 1577
File: the.mulligan.(2022).eng.1cd, Number of Tokens: 6208
File: auggie.rose.(2000).eng.1cd, Number of Tokens: 4611
File: darby.and.joan.s01.e08.episode.1.8.(2022).eng.1cd, Number of Tokens: 2197
File: record.of.ragnarok.s02.e09.resonance.(2023).eng.1cd, Number of Tokens: 898
File: dallas.s01.e03.the.price.you.pay.(2012).eng.1cd, Number of Tokens: 2238
File: sound.of.silence.(2023).eng.1cd, Number of Tokens: 922
File: pepsi.wheres.my.jet.s01.e01.the.kid.from.seattle.(2022).eng.1cd, 

File: yowamushi.pedal.s04.e06.sohoku.shaken.(2018).eng.1cd, Number of Tokens: 1855
File: gangs.of.london.s02.e08.episode.2.8.(2022).eng.1cd, Number of Tokens: 1242
File: the.jeffersons.s07.e20.florences.new.job.part.2.(1981).eng.1cd, Number of Tokens: 1518
File: tom.clancys.jack.ryan.s03.e02.old.haunts.(2022).eng.1cd, Number of Tokens: 1974
File: shababnikim.s01.e10.chiuv.(obligation).(2018).eng.1cd, Number of Tokens: 2001
File: free.s03.e02.a.promise.on.a.shooting.star.(2018).eng.1cd, Number of Tokens: 1815
File: st.elsewhere.s03.e21.murder.she.rote.(1985).eng.1cd, Number of Tokens: 3008
File: the.jewel.in.the.crown.s01.e09.the.towers.of.silence.(1984).eng.1cd, Number of Tokens: 2442
File: northern.exposure.s03.e01.the.bumpy.road.to.love.(1991).eng.1cd, Number of Tokens: 5362
File: cash.register.s02.e18.fifty.million.(2020).eng.1cd, Number of Tokens: 1569
File: the.tick.s03.e05.devil.in.diapers.(1996).eng.1cd, Number of Tokens: 1205
File: love.triangle.(au).s01.e08.cinderellas.shoe.(2

File: shameless.s01.e08.its.time.to.kill.the.turtle.(2011).eng.1cd, Number of Tokens: 4921
File: csi.crime.scene.investigation.s02.e23.the.hunger.artist.(2002).eng.1cd, Number of Tokens: 2062
File: fruits.basket.s01.e22.because.i.was.happy.(2019).eng.1cd, Number of Tokens: 1342
File: flightrisk.(2022).eng.1cd, Number of Tokens: 5492
File: mh370.the.plane.that.disappeared.s01.e02.the.hijack.(2023).eng.1cd, Number of Tokens: 2244
File: desolasi.(2016).eng.1cd, Number of Tokens: 2060
File: natural.world.s38.e02.natures.biggest.beasts.(2018).eng.1cd, Number of Tokens: 1973
File: sayapsayap.patah.(2022).eng.1cd, Number of Tokens: 2317
File: american.horror.story.s11.e04.black.out.(2022).eng.1cd, Number of Tokens: 2015
File: samurai.rabbit.the.usagi.chronicles.s02.e08.the.chizu.stands.alone.(2022).eng.1cd, Number of Tokens: 980
File: my.name.is.earl.s01.e24.number.one.(2006).eng.1cd, Number of Tokens: 1952
File: friends.s09.e21.the.one.with.the.fertility.test.(2003).eng.1cd, Number of Tokens

...

File: industry.s02.e07.lone.wolf.and.cub.(2022).eng.1cd, Number of Tokens: 3748
File: the.good.doctor.s06.e05.growth.opportunities.(2022).eng.1cd, Number of Tokens: 2084
File: so.help.me.todd.s01.e16.twelve.worried.persons.(2023).eng.1cd, Number of Tokens: 3326
File: la.traviata.(2005).eng.1cd, Number of Tokens: 2667
File: mom.s06.e17.a.dark.closet.and.therapy.with.horses.(2019).eng.1cd, Number of Tokens: 2512
File: teen.wolf.s01.e04.magic.bullet.(2011).eng.1cd, Number of Tokens: 1596
File: under.the.vines.s02.e06.episode.2.6.(2023).eng.1cd, Number of Tokens: 2072
File: eva.the.owlet.(2023).eng.1cd, Number of Tokens: 1678
File: chucky.s02.e04.death.on.denial.(2022).eng.1cd, Number of Tokens: 2822
File: panhandle.s01.e05.one.flew.over.the.peacocks.nest.().eng.1cd, Number of Tokens: 2921
File: kyuketsuki.sugu.shinu.s01.e05.the.longest.day.of.tsujigiri.nagiricats.are.cutehanda.day.off.history.(2021).eng.1cd, Number of Tokens: 2215
File: being.human.s01.e09.i.want.you.back.(from.the.dead).

File: monster.s01.e21.happy.holidays.(2004).eng.1cd, Number of Tokens: 1341
File: vidas.menores.(2020).eng.1cd, Number of Tokens: 3347
File: futurama.s08.e04.law.and.oracle.(2011).eng.1cd, Number of Tokens: 1393
File: criminal.minds.s09.e21.what.happens.in.mecklinburg.(2014).eng.1cd, Number of Tokens: 4479
File: khorfakkan.(2020).eng.1cd, Number of Tokens: 3535
File: pioneers.of.television.s01.e03.variety.(2008).eng.1cd, Number of Tokens: 3580
File: sawed.off.(2022).eng.1cd, Number of Tokens: 3341
File: vesper.(2022).eng.1cd, Number of Tokens: 1491
File: gossip.girl.s02.e04.the.exfiles.(2008).eng.1cd, Number of Tokens: 2429
File: mob.psycho.100.s02.e05.discord.choices.(2019).eng.1cd, Number of Tokens: 2009
File: veep.s07.e06.oslo.(2019).eng.1cd, Number of Tokens: 2922
File: savage.river.s01.e01.episode.1.1.(2022).eng.1cd, Number of Tokens: 1936
File: teen.wolf.s05.e16.lie.ability.(2016).eng.1cd, Number of Tokens: 1320
File: a.different.world.s04.e18.the.cash.isnt.always.greener.(1991).

File: the.marvelous.mrs.maisel.s05.e01.episode.5.1.(2023).eng.1cd, Number of Tokens: 4311
File: somebody.feed.phil.s01.e03.tel.aviv.(2018).eng.1cd, Number of Tokens: 4056
File: studio.one.s07.e01.twelve.angry.men.(1954).eng.1cd, Number of Tokens: 3150
File: ncis.s03.e10.probie.(2005).eng.1cd, Number of Tokens: 2778
File: ginger.snaps.back.the.beginning.(2004).eng.1cd, Number of Tokens: 2154
File: vikings.valhalla.s02.e04.the.thaw.(2023).eng.1cd, Number of Tokens: 1057
File: charmed.s01.e03.sweet.tooth.(2018).eng.1cd, Number of Tokens: 2507
File: country.music.s01.e05.the.sons.and.daughters.of.america.(1964.1968).(1968).eng.1cd, Number of Tokens: 7878
File: the.flash.s09.e08.partners.in.time.(2023).eng.1cd, Number of Tokens: 2468
File: a.kidnap.().eng.1cd, Number of Tokens: 3201
File: bleach.s06.e15.clash.the.black.bankai.and.the.white.bankai.(2007).eng.1cd, Number of Tokens: 1334
File: alias.s03.e08.breaking.point.(2003).eng.1cd, Number of Tokens: 1873
File: perfect.strangers.s02.e19.s

File: american.experience.s07.e02.fdr.part.ii.(1994).eng.1cd, Number of Tokens: 3998
File: the.equalizer.s02.e21.hand.and.glove.(1987).eng.1cd, Number of Tokens: 1979
File: between.the.world.and.us.s01.e03.bluff.(2022).eng.1cd, Number of Tokens: 1977
File: workin.moms.s07.e01.ohmygodohmygodohmygod.(2023).eng.1cd, Number of Tokens: 1615
File: doctor.who.confidential.s05.e09.what.goes.on.tour.(2010).eng.1cd, Number of Tokens: 792
File: vacaciones.de.terror.2.(1991).eng.1cd, Number of Tokens: 1590
File: dexter.s03.e01.our.father.(2008).eng.1cd, Number of Tokens: 2809
File: the.sopranos.s06.e02.join.the.club.(2006).eng.1cd, Number of Tokens: 2531
File: alias.s05.e12.theres.only.one.sydney.bristow.(2006).eng.1cd, Number of Tokens: 2223
File: the.madame.blanc.mysteries.s02.e07.episode.2.7.(2023).eng.1cd, Number of Tokens: 2216
File: hot.in.cleveland.s05.e09.bad.george.clooney.(2014).eng.1cd, Number of Tokens: 1521
File: duty.after.school.s01.e01.question.the.spheres.(2023).eng.1cd, Number of

File: balthazar.s04.e07.souvienstoi.(2022).eng.1cd, Number of Tokens: 1659
File: my.life.is.murder.s03.e09.staying.mum.().eng.1cd, Number of Tokens: 3136
File: fringe.s03.e07.the.abducted.(2010).eng.1cd, Number of Tokens: 2072
File: jackie.chan.adventures.s01.e02.the.power.within.(2000).eng.1cd, Number of Tokens: 769
File: shakespeare.uncovered.s02.e06.romeo.juliet.with.joseph.fiennes.(2015).eng.1cd, Number of Tokens: 3420
File: american.experience.s21.e09.we.shall.remain.part.v.wounded.knee.(2009).eng.1cd, Number of Tokens: 4621
File: calling.nurse.meow.(2022).eng.1cd, Number of Tokens: 3250
File: the.sixth.secret.(2022).eng.1cd, Number of Tokens: 4246
File: outlander.s01.e01.sassenach.(2014).eng.1cd, Number of Tokens: 2248
File: nightman.s01.e16.nightwoman.(1998).eng.1cd, Number of Tokens: 2245
File: the.night.agent.s01.e02.redial.(2023).eng.1cd, Number of Tokens: 2670
File: according.to.jim.s07.e18.the.devil.went.down.to.oak.park.(2008).eng.1cd, Number of Tokens: 1440
File: footnote

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Define the document chunker function
def document_chunker(document, chunk_size=1500, overlap_size=200):
    tokens = word_tokenize(document)  # Tokenize the document
    num_tokens = len(tokens)
    chunks = []

    # Iterate over the tokens and create chunks with specified overlap
    for start in range(0, num_tokens, chunk_size - overlap_size):
        end = min(start + chunk_size, num_tokens)
        chunk = tokens[start:end]
        chunks.append({'name': row['name'], 'chunk_index': len(chunks) + 1, 'chunk_text': ' '.join(chunk)})

    return chunks

In [ ]:
# Define the document chunker function
def document_chunker(document, chunk_size=1500, overlap_size=200):
    tokens = word_tokenize(document)  # Tokenize the document
    num_tokens = len(tokens)
    chunks = []

    # Iterate over the tokens and create chunks with specified overlap
    for start in range(0, num_tokens, chunk_size - overlap_size):
        end = min(start + chunk_size, num_tokens)
        chunk = tokens[start:end]
        chunks.append({'name': row['name'], 'chunk_index': len(chunks) + 1, 'chunk_text': ' '.join(chunk)})

    return chunks

In [ ]:
# Apply the document chunker to each cleaned document in the DataFrame
all_chunks = []
for index, row in df.iterrows():
    document = row['cleaned_content']
    document_chunks = document_chunker(document)
    all_chunks.extend(document_chunks)

# Create a new DataFrame from the list of chunks
chunks_df = pd.DataFrame(all_chunks)

In [ ]:
print(chunks_df.shape)

chunks_df.head()

(10256, 3)


,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [ ]:
chunks_df.to_csv("chunks_data.csv",index=False)

### **3. Experiment with the following to generate text vectors of subtitle documents:**

a. BOW/TFIDFto generate sparse vector representations. Note that this will only help you to build a Keyword Based Search Engine.

b. BERT based “SentenceTransformers” to generate embeddings which encode
 semantic information. This can help us build a Semantic Search Engine.

In [ ]:
df1 = pd.read_csv(r"C:\Users\HOME\Downloads\chunks_data.csv")

print(df1.shape)

df1.head()

(10256, 3)


,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [ ]:
df1['chunk_text'][111]

'advertise product brand contact wwwopensubtitlesorg today congratulation rabbi oh well rabbi wife asks tell im lasri send father warm gratitude chandelier beautiful im asked right educational method yeshiva headmaster turned complained yeshiva student went ski vacation ski vacation five day complained ignore keep theyre good men study excel wanted day break expel yeshiva unacceptable every time student finish mishna chapter theyll go trip school would stay empty gmara book would left unread corner torah headmaster asked correct educational method teach boy way impose mountain basin answer ill tell right method right method daniel paran dori medium darset present created written directed eliran malka creator main editor daniel paran directed maor zaguri new black gedaliah stop bugging sorry rabbi come need come well yesterday awesome im gedaliah hi im talus usually break ice begin date story first date rabbi ovadia wife told information im going study torah wont clean wont cook wont an

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Calculate TF-IDF for the documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df1['chunk_text'])

In [ ]:
query="thats key hard tell jeromelike delivery hold back growth literally nightmare scenario college debt country would eighth wealthiest country world kevin connell falling almost student debt wrote two book exposing corruption student debt system time one word would describe student debt system predatory conditioning student face entire life saying must go college order successful people willing pay whatever take go college prominent economist agree economist prominent economist shoot good overlapping chatter get prominent economist big problem would say oh immoral problem impose upon young people narrative tell go school better college key mobility young person year old making authentic choice well feel like ive done everything right growing know im told go school work hard whatever want whatever want kinda like okay um know im happened talked people different community across country see student debt affected adult would tell go college like college degree would mean getting goodpaying job buy house live life want know seems silly imagine served eight year united state air force kind like supposed contributed country yet student loan put debt thought would avoided joining military supposed thought achieving american dream american dream mean thing american dream zero dollar lot people mean many people negative net worth cant reach american dream zero dollar dont worry medium back almost college student using student loan pay spring break trip important learn lesson early personal responsibility pull bootstrap adult talking need realize take debt liable repay debt someone might say listen took loan pay back simple like dont irresponsible say think would said totally make sense eight year ago let forget kid told sign dotted line parental consent necessary 17yearold self unprepared detail medium say thats excuse student knew exactly getting knew getting someone say dont get people dont know theyre actually signing there whole series gimmick played industry attempt cloud actual picture look like lender become creative inflate loan give teaser rate start artificially low thats right college lender using classic first one free move used everything streaming service cocaine im told thats trick sleeve lot fine print overwhelming maybe make underpayment one month pyramid fee dont notice end defaulting wage garnished come say nope taking wage without sort due process thing social security youre teenager signing paper dont listen adult room guide remember guidance counselor reeked coffee pretended know name truly dont think guidance counselor parent family dont know anyone understood true detriment okay blake come bring banana stick banana right foot like yes position way oh like right make sure there nothing else background see foot banana hows real reason came see marcella daughter student debt control running active business creating erotic video help pay every day wake purpose day make sure child better life oldest daughter prepare first four year college started kindergarten young know going go master wasnt prepared know gon na want go law school told encouraged yes okay great need marcellas daughter needed take loan im trying work hard finish gon na wait minute marcella started loan salary going paying student loan right even four year residency im gon na basically debt ive talked people say ive paying student loan year ive making payment ive time yet owe beginning phenomenon compounding interest okay let stop second compounding interest pretty tricky understand here need know loan make pay small amount interest compounding interest charge interest interest making balance grow exponentially quickly get control person student debt may say know gosh really messed get like million people youre like okay maybe systemic maybe isnt couple people made bad decision maybe actually orchestrated scam im convinced system rigged create crisis today thats learned piece puzzle got little attention would change everything could point one thing student debt system one thing would change would bankruptcy mean intent impossible discharge student loan bankruptcy wait thought debt could discharged bankruptcy except student loan essentially took bankruptcy protection permanently away time federal student loan could true isnt bankruptcy bad thing think first time heard word bankrupt watching wheel fortune audience go aw lose money assumed thats something dont want well ive many client postbankruptcy go buy house theyve never able ive two client actually become multimillionaires hard would discharge student loan debt bankruptcy impossible guess bankruptcy reserved truly neediest case right fair deserve fresh start mostly student borrower deserve option doesnt mean everyones going run file bankruptcy even there something much bigger play absence threat bankruptcy open door widespread abuse lender without bankruptcy option college lender frivolously lend much money anybody want apply repayment guaranteed student pay back matter bankruptcy option lender know cant run cant hide essentially give license steal lender incentive restrict theyre lending much loan guess happens tuition cost college raised price essentially youre paying college lender know obscene tuition ridiculous gimmick college play richer college lender become threat bankruptcy force lender behave rationally modicum good faith without bankruptcy protection left selfperpetuating system greed know outset legislatively manufactured crisis reauthorization higher education act senate vote adoption conference report accompanying hr6 mr biden mr bingaman mr durbin mr feingold mr feinstein mr ford like uproar happened like big headline time headline fact dont think congressman even knew happened probably week month maybe even year afterwards bill sign moment enhance economic strength america massive change economy would affect people one noticed sneaky two line slipped conference committee dark night two line legal mumbo jumbo would alter economy multiple generation almost entirely technically bankruptcy code say represents undue hardship get discharge undue hardship chuckle shouldnt hard prove well court interpreted saying mean there hope repaying mean there supposed hope condition youd never want get student loan discharged mean kind condition well might serious cancer physical disability let understand way get rid student debt right cancer thats probably best way well unfortunately would help okay responsible craziness there name find attached nobody seems willing step take credit wait wait wait nobody put name dontwe dont actually know like whisper wind dont know something bear repeating according alan everything ballooning student debt spiraling college cost traced back one twoline provision law"

In [ ]:
 def clean_subtitle(subtitle):
    # Implement your subtitle cleaning logic here
    # For example, you might remove special characters, lowercase the text, remove stopwords, etc.
    cleaned_subtitle = subtitle.lower()  # Example: Convert subtitle to lowercase
    return cleaned_subtitle

In [ ]:

cleaned_query = clean_subtitle(query)

# Calculate TF-IDF for the query
query_tfidf = tfidf_vectorizer.transform([cleaned_query])

# Calculate cosine similarity between the query and documents
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

# Get the indices of documents sorted by similarity score
sorted_indices = cosine_similarities.argsort()[0][::-1]

In [ ]:

# Return top 5 most similar chunk_text
top_k = 5
top_documents = df1.loc[sorted_indices[:top_k], 'chunk_text'].tolist()
print("Top 5 most similar chunk_text:")
for i, text in enumerate(top_documents, 1):
    print(f"Document {i}: {text}")

Top 5 most similar chunk_text:
Document 1: quiet theme music patriotic music america crowd chatter land opportunity basic promise nation reward effort determination every single american least among u shall equal chance achieve greatest thing door higher education open matter circumstance part life great nation one thing president agree unique way patriotic music wind good life see soon get good life follow instruction manual right thing study hard nice brother get go college graduate voila achieve american dream point clattering u economy shrinking theyre getting married theyre getting home theyre getting job weve looking year flat wage door opportunity got harder open light laughter big one staggering debt student loan today old instruction manual feel like false advertising system unfair tell young people go college take loan order graduate theyre punished right thing borrowing money right reason higher education tense music story basic bargain broken greed politics two line slipped

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
df1=pd.read_csv("chunks_data.csv")
print(df1.shape)
df1.head()

(10256, 3)


,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [ ]:
def get_embeddings(text):
    return model.encode(text)

In [ ]:
# Encode the chunk text to get semantic embeddings
chunk_embeddings = model.encode(df1['chunk_text'].tolist())

# Add the embeddings to the DataFrame
df1['embeddings'] = chunk_embeddings.tolist()

In [ ]:
df1.shape

### **5. Store embeddings in a ChromaDB database.**

In [ ]:
import chromadb

In [ ]:
client=chromadb.Client()

In [ ]:
collection = client.create_collection(
        name="searchengine",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [ ]:
for i, embedding in enumerate(chunk_embeddings):
    collection.add(ids=f"chunk_{i}",  # Replace with a more descriptive ID scheme if needed
                   embeddings=embedding.tolist(),  # Convert to a list for Chroma
                   documents=df1.loc[i, "chunk_text"],  # Use chunk_text as documents
                   metadatas={"name": df1.loc[i, "name"]}  # Use 'name' column data as metadata
    )


# **Part 2: Retrieving Documents**

**1. Take the user's search query.**

**2. Preprocess the query (if required).**

**3. Create query embedding.**

**4. Using cosine distance, calculate the similarity score between embeddings of documents and user search query embedding.**

**5. These cosine similarity scores will help in returning the most relevant candidate documents as per user's search query.**

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
subtitle_text = "help kiss excuse im gon na kiss need facial facial ouch thats gon na hurt tell straight english dont want wear wont force try okay go ahead dont like doesnt unlike doesnt like unli unlike fine give like doesnt dont right stay still sir tep really hurt love miss monica remember know say love hurt wow looking good hmm think look good mmm modern cool whats last one posh posh posh mama mae yeah ill posh mmhmm let posh mama mae ow ow im sorry hurt done bit ow ooh ow hand hurt sorry sorry sir please hold still look yuck dinner fork good salad fork good mmm okay "
cleaned_subtitle = clean_subtitle(subtitle_text)

In [ ]:

# Encode the cleaned subtitle text to get semantic embeddings
query_embedding = model.encode([cleaned_subtitle])

In [ ]:
# Perform a query on the ChromaDB collection using the embeddings
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
    include=['documents']
)


In [ ]:
documents = results['documents']

# Iterate over the documents and print each document
for i, query_documents in enumerate(documents):
    for j, document in enumerate(query_documents):
        print(f"Document {i+1}, Item {j+1}: {document}")



Document 1, Item 1: teeth last time wed done twice would really embarrassed idve thought oh god one u obviously bad kisser harry oh god im shocked mum youre one million card call boy say special talent pulling milfs feel like annamay deep wanted kiss kai cant lie teddy bear chain saying sergeant snake boy told girl joining army stop seeing rude olivias one bit long like roundup kiss hold one like ouch shaq ever find lied well shes gon na see see weve got hair removal cream boy blistered entire body using hairremoval cream lad holiday right kiss frog find prince fake foot boy claim nan initial tattooed foot really girl slept lad holiday uh oh trouble somethings come along burst bubble ive got menu say porn cocktail chicken tikka massageher dessert ice makeherscream oh god thats vibe wow boy caught sex restaurant waiter oh feel like ive kissed boy honestly think kissed challenge girl cleaned well actually theyve cleaned girl back villa shaqs realising come tanya he got major feeling coin